     |████████████████████████████████| 204.2MB 70kB/s 
     |████████████████████████████████| 204kB 57.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=e0c3fe538000295e29ed9a8853b11dccffbb91edc412d471607b7fd4a322c510
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import (RandomForestClassifier,
                                       GBTClassifier,DecisionTreeClassifier)

In [ ]:
spark=SparkSession.builder.appName('trees_1').getOrCreate()

In [ ]:
data=spark.read.csv('/content/sample_data/College.csv',inferSchema=True,header=True)

In [ ]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [ ]:
data.head(5)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60),
 Row(School='Adelphi University', Private='Yes', Apps=2186, Accept=1924, Enroll=512, Top10perc=16, Top25perc=29, F_Undergrad=2683, P_Undergrad=1227, Outstate=12280, Room_Board=6450, Books=750, Personal=1500, PhD=29, Terminal=30, S_F_Ratio=12.2, perc_alumni=16, Expend=10527, Grad_Rate=56),
 Row(School='Adrian College', Private='Yes', Apps=1428, Accept=1097, Enroll=336, Top10perc=22, Top25perc=50, F_Undergrad=1036, P_Undergrad=99, Outstate=11250, Room_Board=3750, Books=400, Personal=1165, PhD=53, Terminal=66, S_F_Ratio=12.9, perc_alumni=30, Expend=8735, Grad_Rate=54),
 Row(School='Agnes Scott College', Private='Yes', Apps=417, Accept=349, Enroll=137, Top10perc=60, Top25perc=89, F_Undergrad=510, P

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [ ]:
assembler=VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [ ]:
output=assembler.transform(data)

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer=StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [ ]:
output_fixed=indexer.fit(output).transform(output)

In [ ]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [ ]:
final_data=output_fixed.select('features','PrivateIndex')

In [ ]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
dtc=DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc=RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
gbt=GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

In [ ]:
dtc_model=dtc.fit(train_data)
rfc_model=rfc.fit(train_data)
gbt_model=gbt.fit(train_data)

In [ ]:
dtc_preds=dtc_model.transform(test_data)
rfc_preds=rfc_model.transform(test_data)
gbt_preds=gbt_model.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
my_eval=BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [ ]:
print('DTC AUC :',my_eval.evaluate(dtc_preds))

DTC AUC : 0.9471533764367815


In [ ]:
print('RFC AUC :',my_eval.evaluate(rfc_preds))

RFC AUC : 0.9810524425287356


In [ ]:
print('GBT AUC :',my_eval.evaluate(gbt_preds))

GBT AUC : 0.9811422413793103
